In [2]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utils


In [3]:
train = pd.read_csv(f'./../../Dataset/food_train.tsv', index_col='Index', encoding='utf-8', sep='\t')
X_train = train.drop(columns = 'target')

y_train = train.target
del train
X_test = pd.read_csv(f'./../../Dataset/food_X_test.tsv', index_col='Index', encoding='latin', sep='\t')
y_test_submission_example = pd.read_csv(f'./../../Dataset/y_test_submission_example.tsv', index_col='Index', encoding='utf-8', sep='\t')

In [4]:
process_categories = utils.process_categories()

In [5]:
for df in [X_train,X_test]:
    df['serving_size2'] = df.serving_size.apply(process_categories.get_quantity)
    df['countries2'] = df['countries'].apply(process_categories.process_countries)
    df['region_code2'] = df[['countries','countries_en']].apply(process_categories.get_region_code,axis=1)


In [6]:
X_train,X_test = utils.clean_null(X_train,X_test,0.5)
# remove brands_tags
X_train.drop(columns =['brands_tags'],inplace= True)
#X_test.drop(columns =['brands_tags'],inplace= True)


Train init shape  (102028, 60)
Test  init shape  (25508, 60)
has  82.84686556631513 % null elements -> deleting .. generic_name
has  70.76684831614851 % null elements -> deleting .. quantity
has  74.81181636413532 % null elements -> deleting .. packaging
has  74.8127964872388 % null elements -> deleting .. packaging_tags
has  70.80507311718351 % null elements -> deleting .. categories
has  70.80507311718351 % null elements -> deleting .. categories_tags
has  70.80507311718351 % null elements -> deleting .. categories_en
has  93.74877484612068 % null elements -> deleting .. origins
has  93.75269533853452 % null elements -> deleting .. origins_tags
has  87.8680362253499 % null elements -> deleting .. manufacturing_places
has  87.8680362253499 % null elements -> deleting .. manufacturing_places_tags
has  82.01964166699337 % null elements -> deleting .. labels
has  81.98729760457914 % null elements -> deleting .. labels_tags
has  81.98729760457914 % null elements -> deleting .. labels_en
h

In [7]:
X_train_embeddings = pd.read_csv(f'./../../Dataset/train_embeddings.csv',encoding='utf-8')
X_test_embeddings = pd.read_csv(f'./../../Dataset/train_embeddings.csv', encoding='utf-8')

In [8]:
X_train.head(3)

,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,brands,countries,countries_tags,countries_en,...,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,states,states_tags,states_en,serving_size2,countries2,region_code2
Index,,,,,,,,,,,,,,,,,,,,,
104815,usda-ndb-import,1489135595,2017-03-10T08:46:35Z,1489135595,2017-03-10T08:46:35Z,Caramel Creams,Gold Emblem,US,en:united-states,United States,...,"en:e322,en:e322i","E322 - Lecithins,E322i - Lecithin",0.0,0.0,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",34.0,US,19.0
62513,usda-ndb-import,1489069556,2017-03-09T14:25:56Z,1489069556,2017-03-09T14:25:56Z,"Potato Chips, Hot Bar-B-Q",Conn's,US,en:united-states,United States,...,en:e621,E621 - Monosodium glutamate,0.0,0.0,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,US,19.0
111040,usda-ndb-import,1489061137,2017-03-09T12:05:37Z,1489061138,2017-03-09T12:05:38Z,Hummus,Sticks Kebob Shop,US,en:united-states,United States,...,NaN,NaN,0.0,0.0,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",57.0,US,19.0


In [9]:
X_train_embeddings.head(2)

,creator,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,...,emb_product_name_ingredients_text_brands_tags_additives__758,emb_product_name_ingredients_text_brands_tags_additives__759,emb_product_name_ingredients_text_brands_tags_additives__760,emb_product_name_ingredients_text_brands_tags_additives__761,emb_product_name_ingredients_text_brands_tags_additives__762,emb_product_name_ingredients_text_brands_tags_additives__763,emb_product_name_ingredients_text_brands_tags_additives__764,emb_product_name_ingredients_text_brands_tags_additives__765,emb_product_name_ingredients_text_brands_tags_additives__766,emb_product_name_ingredients_text_brands_tags_additives__767
0,usda-ndb-import,caramel creams,NaN,NaN,NaN,NaN,gold emblem,gold emblem,NaN,NaN,...,0.396211,-0.234920,0.627098,0.107169,0.025944,0.031931,0.211013,-0.321392,-0.327091,-0.818351
1,usda-ndb-import,"potato chips, hot bar b q",NaN,NaN,NaN,NaN,conn's,conn s,NaN,NaN,...,1.445435,-0.103333,0.860991,1.599039,0.660110,0.136999,0.554076,-0.890237,-0.126865,-0.367392


In [10]:
for df in [X_train,X_test]:
    df.reset_index(inplace=True)

In [11]:
total_train = pd.concat([X_train[['region_code2','countries2','serving_size2']], X_train_embeddings], axis=1)

In [12]:
total_test = pd.concat([X_test[['region_code2','countries2','serving_size2']], X_test_embeddings], axis=1)

In [13]:
y_train = total_train.target
y_test = total_test.target
for df in [total_train,total_test]:
    df.drop(columns='target',inplace=True)
    #target_bin


In [14]:
None

In [15]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [40]:

for df in [total_train,total_test]:
    df.drop(columns='target_bin',inplace=True)
    #target_bin


In [16]:
for df in [total_train,total_test]:
    df.replace(np.nan, 0)


In [71]:
train_numeric_desc,train_categorical_desc = utils.my_df_describe(total_train,'train',show=False)
test_numeric_desc,test_categorical_desc = utils.my_df_describe(total_test,'test',show=False)


********************train********************
shape  (102028, 857)
********************test********************
shape  (102028, 857)


In [18]:
datatype = total_train.dtypes
tmp3 = datatype[(datatype == 'object') | (datatype == 'category')].index.tolist()

In [19]:
for df in [total_train,total_test]:
    df = df.loc[:,~df.columns.duplicated()]

In [20]:
None

from sklearn.preprocessing import OneHotEncoder


from sklearn import preprocessing

data_total = pd.concat([total_train,total_test])
for name in train_categorical_desc.index:
    data_total[name] = pd.get_dummies(data_total[name], prefix='')
    #trained_le = le.fit(data_total[name])
    #total_train[name]= trained_le.transform(total_train[name])
    #total_test[name] = trained_le.transform(total_test[name])

from sklearn import preprocessing

data_total = pd.concat([total_train,total_test])
for name in train_categorical_desc.index:
    print(name)
    le = preprocessing.LabelEncoder()
    trained_le = le.fit(data_total[name])
    total_train[name]= trained_le.transform(total_train[name])
    total_test[name] = trained_le.transform(total_test[name])
    

In [26]:
train_numeric_desc.index

Index(['region_code2', 'serving_size2', 'cities', 'allergens_en',
       'no_nutriments', 'additives_n', 'ingredients_from_palm_oil_n',
       'ingredients_from_palm_oil', 'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_that_may_be_from_palm_oil',
       ...
       'emb_product_name_ingredients_text_brands_tags_additives__758',
       'emb_product_name_ingredients_text_brands_tags_additives__759',
       'emb_product_name_ingredients_text_brands_tags_additives__760',
       'emb_product_name_ingredients_text_brands_tags_additives__761',
       'emb_product_name_ingredients_text_brands_tags_additives__762',
       'emb_product_name_ingredients_text_brands_tags_additives__763',
       'emb_product_name_ingredients_text_brands_tags_additives__764',
       'emb_product_name_ingredients_text_brands_tags_additives__765',
       'emb_product_name_ingredients_text_brands_tags_additives__766',
       'emb_product_name_ingredients_text_brands_tags_additives__767'],
      dtype='ob

In [72]:
from sklearn.model_selection import train_test_split

X_train2, X_test2, y_train2, y_test2 = train_test_split(total_train[train_numeric_desc.index], y_train, test_size=0.3, random_state=42)

In [83]:
import lightgbm as lgb

hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['rmse'],
    'learning_rate': 0.005,
    "num_leaves": 128,  
    "max_bin": 512,
}

hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'eval_metric':'rmse',
    'metric': ['rmse'],
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 5000,
    "n_estimators": 200
}
lgb_train = lgb.Dataset(X_train2, y_train2)#categorical_feature= list(train_categorical_desc.index))
lgb_test = lgb.Dataset(X_test2, y_test2)#,categorical_feature= list(train_categorical_desc.index))

In [84]:
bst = lgb.train(hyper_params, lgb_train, valid_sets=lgb_test, early_stopping_rounds=100, verbose_eval=True)     

bst.save_model('model.txt', num_iteration=bst.best_iteration)

[52]	valid_0's rmse: 4.60677
[53]	valid_0's rmse: 4.60294
[54]	valid_0's rmse: 4.59917
[55]	valid_0's rmse: 4.59663
[56]	valid_0's rmse: 4.59395
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[57]	valid_0's rmse: 4.59184
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[58]	valid_0's rmse: 4.58973
[59]	valid_0's rmse: 4.58789
[60]	valid_0's rmse: 4.58567
[61]	valid_0's rmse: 4.58387
[62]	valid_0's rmse: 4.58247
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[63]	valid_0's rmse: 4.58102
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[64]	valid_0's rmse: 4.58026
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[65]	valid_0's rmse: 4.57923
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[66]	valid_0's rmse: 4.57868
[67]	valid_0's rmse: 4.57805
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[68]	valid_

In [79]:
y_train2_preds = bst.predict(X_train2)
y_test2_preds = bst.predict(X_test2)

rmse = np.sqrt(mean_squared_error(y_train2, y_train2_preds))
print("Train RMSE: %f" % (rmse))
rmse = np.sqrt(mean_squared_error(y_test2, y_test2_preds))
print("Test RMSE: %f" % (rmse))


Train RMSE: 4.118005
Test RMSE: 4.576333
